# Automated ML

Done: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

## Dataset

### Overview
I am using the "Jobs and SAlaries in Data Science" dataset from kaggle (https://www.kaggle.com/datasets/hummaamqaasim/jobs-in-data?resource=download). It contains job descriptions (e.g., Data Engineer, Data Scientist) and salaries (both in original currency and USD), experience level, type of employment (full time vs part-time), and work setting (in-person/hybrid). For the capstone project I want to predict the salary in USD using a regression model. It's important to delete the "salary" column (in original currency) beforehand, as it is almost the same as the target column and should not be an input of the model.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'my-automl-experiment'

experiment=Experiment(ws, experiment_name)



In [ ]:
found = False
key = "DataScientistSalaries"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/sarahsofia93/udacity-MLengineer-capstone/blob/master/jobs_in_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()



In [ ]:
amlcompute_cluster_name = "my-fancy-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

## AutoML Configuration

(Done): Explain why you chose the automl settings and cofiguration you used below.

I am choosing a regression model since my target variable (salary) is a continuous rather than a categorial variable. I am using the primary metric R2 score which is a common metric for regresisons in AutoML. It also helps me to establish comparability with the hyperdrive model, where I also use the r aquared as metric.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'r2_score',
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                             task = "regression",
                                training_data = dataset,
                                label_column_name = "salary_in_usd",
                                path = "./automl-run",
                                enable_early_stopping= True,
                                featurization = "auto",
                                debug_log = "automl_errors.log",
                                **automl_settings
)


In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_model, best_model_metrics = remote_run.get_output()
best_model.get_properties()
print(best_model)
best_model_metrics = best_model.get_metrics()

for metric_name in best_model_metrics:
    metric = best_model_metrics[metric_name]
    print(metric_name, metric)



In [ ]:
#TODO: Save the best model
import joblib
os.makedirs('./outputs', exist_ok=True)
joblib.dump(best_model, filename='outputs/best_automl_model.joblib')

myModel = best_model.register_model(model_name='best_automl_model', model_path='outputs/model.pkl')


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Download scoring file 
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_model.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

from azureml.automl.core.shared import constants
best_model.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core import Model

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_model.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import json


# df is the data as seen by the model during training
df = pd.read_csv('jobs_in_data.csv')
df = df.drop(columns=['salary', 'currency'])


test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('Class')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)


In [ ]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)


In [ ]:
# Print results from the inference
print(response.text)
# Print original labels
print(label_df)

print(service.get_logs())

service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
